导入包

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import warnings

设置(UA,地点,距离),并忽略警告(来自beautifulfoup)

In [2]:
warnings.filterwarnings('ignore')
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36', 
             'Connection': 'keep-alive'}
long1 = 117.224817
latitude1 = 31.825842
distance1 = 6
num1 = 1596451211625
long = 117.235447
latitude = 31.82687
distance = 10
num = 1597242864313

~~获取代理(**已弃用**)~~

In [3]:
def getProxy():
    url = 'http://127.0.0.1:59977/getip?price=0&word=&count=10&type=json&detail=true'
    r = requests.get(url)
    proxys = json.loads(r.text)
    proxyList = []
    for i in proxys:
        print("IP:{}\tPort:{}\tOut IP:{}\tArea:{}".format(i['ip'], i['port'], i['out_ip'], i['area_text']))
        proxyList.append({'http':'http://{}:{}'.format(i['ip'], i['port'])})
    return proxyList

加载cookie(由Chrome插件EditThisCookie导出为json格式)

In [4]:
def loadCookie(fileName):
    cookie = requests.cookies.RequestsCookieJar()
    with open(fileName, 'r') as fp:
        data = json.load(fp)
    for i in data:
        cookie.set(i['name'], i['value'], domain=i['domain'], path=i['path'])
    return cookie

从天眼查获取一定区域内的公司信息(参数从网页请求参数中获取)

In [16]:
def getAllCompany(longtitude, latitude, distance, num, status = None):
    global cookie
    companyNum = 0
    result = []
    with open('{}-{}-{}-{}'.format(longtitude, latitude, distance, num), 'w') as fp:
        count = 0
        for page in range(1, 251):
            if status == None:
                url = 'https://www.tianyancha.com/map/getListByPoint.html?pageNum={}&longtitude={}&latitude={}&distance={}&_={}'.format(page, longtitude, latitude, distance, num)
            else:
                url = 'https://www.tianyancha.com/map/getListByPoint.html?pageNum={}&longtitude={}&latitude={}&distance={}&regStatus={}&_={}'.format(page, longtitude, latitude, distance, status, num)
            while count < 5:
                try:
                    r = requests.get(url, cookies = cookie, headers = headers)
                    count = 0
                    break
                except:
                    print('Retry: {}/5'.format(count))
                    count += 1
            if count == 5:
                raise(Exception, "failed to connect")
            soup = BeautifulSoup(r.text, 'html.parser')
            tags = soup.find_all('td')
            value = []
            count = 0
            for j in tags:
                count += 1
                value.append(j.string)
                if count%8 == 0:
                    #print(allCount, value)
                    result.append({'name':value[1], 'regCurrent':value[3], 'regDate':value[4],\
                                   'type':value[6], 'status':value[7]})
                    value = []
            print('{}/250\tget {} company'.format(page, len(result) - companyNum))
            companyNum = len(result)
        json.dump(result, fp)

~~处理爬取的公司信息(适配于getAllCompany)(**已整合进getAllCompany**)~~

In [6]:
def processHTML(fileName):
    with open(fileName, 'r') as fp:
        allHTML = fp.read()
    HTMLList = allHTML.split('**********')
    allCount = 0
    result = []
    for i in HTMLList:
        soup = BeautifulSoup(i, 'html.parser')
        tags = soup.find_all('td')
        value = []
        count = 0
        for j in tags:
            count += 1
            value.append(j.string)
            if count%8 == 0:
                allCount += 1
                #print(allCount, value)
                result.append({'name':value[1], 'regCurrent':value[3], 'regDate':value[4],\
                               'type':value[6], 'status':value[7]})
                value = []
    return result

~~天眼查版查询具体公司注销信息(**已弃用**)~~

In [7]:
def getCompanyByName(name):
    global cookie#, proxyList
    #proxy = proxyList[randint(0, len(proxyList) - 1)]
    url = 'https://www.tianyancha.com/search?key={}'.format(name)
    r = requests.get(url, cookies = cookie) #proxies = proxy)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('a')
    for i in tags:
        #print(i.string)
        if i.string == name:
            #print('find')
            print(i.get('href'))
            url = i.get('href')
            break
    r = requests.get(url, cookies = cookie)#, proxies = proxy)
    print(r.text)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('span')#, class_ = 'sec-c3')
    for i in tags:
        print(i)

企查查版查询具体公司注销信息

In [8]:
def getQCCCompanyByName(name):
    global qccCookie
    url = 'http://www.qcc.com/search?key={}'.format(name)
    r = requests.get(url, cookies = qccCookie, headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('a')
    flag = False
    for i in tags:
        if i.get('href') != None:
            if '/firm/' in i.get('href') and '.shtml' in i.get('href'):
                url = 'https://www.qcc.com' + i.get('href')
                print('Find most closet result: {}'.format(url))
                flag = True
    if flag:
        r = requests.get(url, cookies = qccCookie, headers = headers)
        soup = BeautifulSoup(r.text)
        res = soup.find_all('span', class_ = 'ntag text-danger tooltip-br')
        print(res)
        res = res[0]
        spanTitle = BeautifulSoup(res.get('title'))
        res = spanTitle.find_all('span')
        data = {'date':'', 'reason':''}
        data['date'] = res[0].string[5:]
        data['reason'] = res[1].string[5:]
        return True, data
    else:
        return False, None

~~测试IP(随getProxy弃用)~~

In [9]:
def ipTest():
    global proxyList
    for proxy in proxyList:
        print(proxy)
        r = requests.get('http://icanhazip.com', headers = headers, proxies = proxy)
        print(r.text)

分析注销公司

In [10]:
def resultAnalyze(result):
    companyNum = 0
    notExistNum = 0
    typeNotExistCompany = {}
    typeCompany = {}
    for i in result:
        if i['status'] != '-':
            companyNum += 1
            if i['type'] not in typeCompany.keys():
                typeCompany[i['type']] = 1
            else:
                typeCompany[i['type']] += 1
            if i['status'] != '存续':
                notExistNum += 1
                if i['type'] not in typeNotExistCompany.keys():
                    typeNotExistCompany[i['type']] = 1
                else:
                    typeNotExistCompany[i['type']] += 1
                print("{}-{}-{}-{}-{}".format(i['name'], i['regCurrent'], i['regDate'], i['type'], i['status']))
    print("Not exist rate: {}({}/{})".format(notExistNum/companyNum, notExistNum, companyNum))
    print(typeNotExistCompany)
    print(typeCompany)

In [19]:
cookie = loadCookie('./cookie')

In [20]:
getAllCompany(long, latitude, distance, num, '注销')

1/250	get 20 company
2/250	get 20 company
3/250	get 20 company
4/250	get 20 company
5/250	get 20 company
6/250	get 20 company
7/250	get 20 company
8/250	get 20 company
9/250	get 20 company
10/250	get 20 company
11/250	get 20 company
12/250	get 20 company
13/250	get 20 company
14/250	get 20 company
15/250	get 20 company
16/250	get 20 company
17/250	get 20 company
18/250	get 20 company
19/250	get 20 company
20/250	get 20 company
21/250	get 20 company
22/250	get 20 company
23/250	get 20 company
24/250	get 20 company
25/250	get 20 company
26/250	get 20 company
27/250	get 20 company
28/250	get 20 company
29/250	get 20 company
30/250	get 20 company
31/250	get 20 company
32/250	get 20 company
33/250	get 20 company
34/250	get 20 company
35/250	get 20 company
36/250	get 20 company
37/250	get 20 company
38/250	get 20 company
39/250	get 20 company
40/250	get 20 company
41/250	get 20 company
42/250	get 20 company
43/250	get 20 company
44/250	get 20 company
45/250	get 20 company
46/250	get 20 compa

In [21]:
with open('{}-{}-{}-{}'.format(long, latitude, distance, num), 'r') as fp:
    result = json.load(fp)

In [22]:
resultAnalyze(result)

安徽中键环境科学研究院有限公司-501万-2017-11-18-研究和试验发展-注销
安徽省徽盛生态园林有限公司-200万-2012-03-16-农业-注销
合肥市蜀山区明婷餐厅-0.005万人民币-2016-12-14-餐饮业-注销
合肥市市直机关印务有限公司国泰分公司---2008-02-21-印刷和记录媒介复制业-注销
安徽中铁商业有限公司合肥第十七客票代售点---2009-06-10-其他服务业-注销
安徽安财会计师事务所（普通合伙）---2013-03-01-商务服务业-注销
合肥市市直机关印务有限公司国华分公司---2007-07-26-印刷和记录媒介复制业-注销
合肥文成劳务有限公司-50万-2006-06-29-居民服务业-注销
安徽巢湖经济开发区真之味餐馆---2019-07-25-餐饮业-注销
合肥市经开区披啥芝恋小吃中心---2019-10-21-批发业-注销
星利保险经纪有限公司合肥第一营业部---2019-08-15-保险业-注销
合肥经济技术开发区头发乱了理发店---2019-09-16-居民服务业-注销
合肥市包河区超裕贸日用百货商行---2020-03-02-零售业-注销
安徽巢湖经济开发区川香冒菜店---2019-06-14-餐饮业-注销
合肥新站区很范花甲鸡店---2019-09-23-餐饮业-注销
安徽巢湖经济开发区巨航门窗加工厂---2019-10-16-金属制品业-注销
安徽巢湖经济开发区一线上花甲店---2019-05-08-餐饮业-注销
安徽巢湖经济开发区锂皮商务信息经营部---2019-04-09-商务服务业-注销
安徽巢湖经济开发区锦金商务信息经营部---2019-04-09-商务服务业-注销
肥西县郡郡饰品店---2020-01-07-零售业-注销
安徽中键环境科学研究院有限公司-501万-2017-11-18-研究和试验发展-注销
安徽省徽盛生态园林有限公司-200万-2012-03-16-农业-注销
合肥市蜀山区明婷餐厅-0.005万人民币-2016-12-14-餐饮业-注销
合肥市市直机关印务有限公司国泰分公司---2008-02-21-印刷和记录媒介复制业-注销
安徽中铁商业有限公司合肥第十七客票代售点---2009-06-10-其他服务业-注销
安徽安财会计师事务所（普通合伙）---2013-03-01-

In [15]:
result

[{'name': '中国力信股份有限公被合肥政务前却营式心',
  'regCurrent': '-',
  'regDate': '2007-01-10',
  'type': '电信、广播电视和卫星传输服务',
  'status': '存续'},
 {'name': '合肥翰林酒店应理有限公被翰林贵宾楼',
  'regCurrent': '-',
  'regDate': '2006-05-23',
  'type': '餐饮业',
  'status': '存续'},
 {'name': '合肥会考科技有限公被',
  'regCurrent': '2000万',
  'regDate': '2019-06-28',
  'type': '研究和试验发展',
  'status': '存续'},
 {'name': '随徽未果莫界生物这药有限公被',
  'regCurrent': '20000万',
  'regDate': '2016-09-07',
  'type': '科技推广和应用服务业',
  'status': '存续'},
 {'name': '随徽未果生物这药有限公被',
  'regCurrent': '138891万',
  'regDate': '2014-03-10',
  'type': '研究和试验发展',
  'status': '存续'},
 {'name': '随徽未果安胞难疗有限公被',
  'regCurrent': '20000万',
  'regDate': '2014-10-15',
  'type': '研究和试验发展',
  'status': '存续'},
 {'name': '随徽未果伦珠生物这药有限公被',
  'regCurrent': '20000万',
  'regDate': '2016-09-07',
  'type': '科技推广和应用服务业',
  'status': '存续'},
 {'name': '随徽未果生物经候集团有限公被',
  'regCurrent': '30000万',
  'regDate': '2015-09-10',
  'type': '研究和试验发展',
  'status': '存续'},
 {'name': '随徽未果达木生物这药有限公被',
  're